\begin{align*}
P(\Omega, A, B, l) &= A + \sum\limits_{k=0}^{\infty} \sum\limits_{m=0}^{k + 1} \dfrac{\Omega^{k - m + 1}A B^m}{(k + 2)!}
\end{align*}

We can split this series into the terms with omega and those without.

\begin{align*}
P(\Omega, A, B, l) = A + \sum\limits_{k=0}^{\infty}\dfrac{A B^{k+1}}{(k + 2)!} + \sum\limits_{k=0}^{\infty} \sum\limits_{m=0}^{k} \dfrac{\Omega^{k - m + 1}A B^m}{(k + 2)!} 
\end{align*}

\begin{align*}
P_0(\Omega, A, B, l) = A + \sum\limits_{k=0}^{\infty}\dfrac{A B^{k+1}}{(k + 2)!} 
\end{align*}

\begin{align*}
P_1(\Omega, A, B, l) = \sum\limits_{k=0}^{\infty} \sum\limits_{m=0}^{k} \dfrac{\Omega^{k - m + 1}A B^m}{(k + 2)!} 
\end{align*}

\begin{align*}
P(\Omega, A, B, l)  = P_0(\Omega, A, B, l) + P_1(\Omega, A, B, l)
\end{align*}

Since $$B^l = 0$$


\begin{align*}
P_0(\Omega, A, B, l) = A + \sum\limits_{k=0}^{l-2}\dfrac{A B^{k+1}}{(k + 2)!} 
\end{align*}



\begin{align*}
P_0(\Omega, A, B, l) = A \sum\limits_{m=0}^{l-1}\dfrac{B^{m}}{(m + 1)!} 
\end{align*}

We can reoder the indices of the last term so that the powers of B are in the outer summation:  $$ k' = k = m$$

\begin{align*}
 P_1(\Omega, A, B, l)
    &=   \sum\limits_{m=0}^{\infty} \left[ \sum\limits_{k'=0}^{\infty} \dfrac{\Omega^{k' + 1}}{(k' + m + 2)!} \right] A B^m
\end{align*}

Since $$B^{l + 1} = 0$$

\begin{align*}
 P_1(\Omega, A, B, l)
    &=   \sum\limits_{m=0}^{l-1} \left[ \sum\limits_{k=0}^{\infty} \dfrac{\Omega^{k + 1}}{(k + m + 2)!} \right] A B^m
\end{align*}

$k = 2k'$

$k = 2k''$

\begin{align*}
 P_1(\Omega, A, B, l)
    &=   \sum\limits_{m=0}^{l-1} \left[ \sum\limits_{k'=0}^{\infty} \dfrac{\Omega^{2k' + 1}}{(2k' + m + 2)!} 
    + \sum\limits_{k'=0}^{\infty} \dfrac{\Omega^{2k'' + 2}}{(2k'' + m + 3)!}
     \right] A B^m 
\end{align*}

\begin{align*}
 P_1(\Omega, A, B, l)
    &=   \sum\limits_{m=0}^{l-1} \left[  \sum\limits_{k'=0}^{\infty} \dfrac{(-1)^k \theta^{2k}\Omega}{(2k' + m + 2)!} 
    +  \sum\limits_{k'=0}^{\infty} \dfrac{(-1)^k \theta^{2k}\Omega^2}{(2k'' + m + 3)!}
     \right] A B^m \\
\end{align*}

\begin{align*}
c_m(\theta) &= \sum\limits_{k=0}^{\infty} \dfrac{(-1)^k \theta^{2k} }{(2k + m + 2)!} \\
\end{align*}

\begin{align*}
 P_1(\Omega, A, B, l)
    &=   \Omega A \sum\limits_{m=0}^{l-1} c_m(\theta) B^m 
    + \Omega^2  A \sum\limits_{m=0}^{l-1} c_{m+1}(\theta) B^m 
\end{align*}

\begin{align*}
P(\Omega, A, B, l) = A \sum\limits_{m=0}^{l-1}\dfrac{B^{m}}{(m + 1)!}    + \Omega A \sum\limits_{m=0}^{l-1} c_m(\theta) B^m 
    + \Omega^2  A \sum\limits_{m=0}^{l-1} c_{m+1}(\theta) B^m 
\end{align*}

\begin{align*}
c_m(\theta) &= \sum\limits_{k=0}^{\infty} \dfrac{(-1)^k \theta^{2k} }{(2k + m + 2)!} \\
\end{align*}

In [1]:
import math
import casadi as ca
import numpy as np

def nilpotent_degree(B, tol=1e-12, max_power=None):
    """
    Compute the nilpotency degree of a square matrix B.

    Parameters
    ----------
    B : (n,n) array‑like
        The matrix to test (NumPy ndarray, casadi.DM, etc.).
    tol : float, optional
        Numerical tolerance: ‖B^k‖_F ≤ tol  ⇒  treat as zero.  Default 1e‑12.
    max_power : int, optional
        Maximum power to test.  Default is n (matrix size); if the
        nilpotency index is larger, the function returns None.

    Returns
    -------
    int  or  None
        ν  – smallest k ≥ 1 with  B^k ≈ 0,
        or None if the matrix is not nilpotent up to max_power.
    """
    B = np.asarray(B)
    assert B.shape[0] == B.shape[1], "B must be square"
    n = B.shape[0]
    if max_power is None:
        max_power = n          # Cayley‑Hamilton ⇒ ν ≤ n for nilpotent matrices

    Pk = B.copy()              # B¹
    for k in range(1, max_power + 1):
        if np.linalg.norm(Pk, ord='fro') <= tol:
            return k
        Pk = Pk @ B            # next power
    return None


# ----------------------------------------------------------------------
# 1.  tiny‑angle Maclaurin (degree 6 ⇒ O(θ⁸) remainder)
# ----------------------------------------------------------------------
def _cm_maclaurin(theta, m):
    """
    Return c_m(θ) by summing terms up to θ⁶.
    Works with SX because the loop is unrolled at Python level.
    """
    theta2 = theta * theta
    term = 1.0 / math.factorial(m + 2)   # k = 0
    acc  = term

    # k = 1 … 3   (adds θ², θ⁴, θ⁶ terms)
    for k in range(1, 4):
        term *= -theta2 / ((2*k + m + 1) * (2*k + m + 2))
        acc  += term
    return acc


# ----------------------------------------------------------------------
# 2.  closed‑form remainder (unchanged from before)
# ----------------------------------------------------------------------
def _taylor_trunc(theta, p: int, odd: bool):
    r = 1 if odd else 0
    poly = ((-1)**p) / math.factorial(2*p + r)
    for j in range(p-1, -1, -1):
        poly = poly * theta**2 + ((-1)**j) / math.factorial(2*j + r)
    return poly * (theta if odd else 1)

def _c_m_closed(theta, m: int):
    p, r = divmod(m, 2)
    if r == 0:
        rem = ca.cos(theta) - _taylor_trunc(theta, p, odd=False)
    else:
        rem = ca.sin(theta) - _taylor_trunc(theta, p, odd=True)
    return ((-1)**(p + 1) / theta**(m + 2)) * rem


def c_m_safe(theta, m, eps=1e-4):
    """
    Symbolic run‑time switch:
       |θ| < eps → Maclaurin   (high relative accuracy)
       else       → closed form
    """
    return ca.if_else(ca.fabs(theta) < eps,
                      _cm_maclaurin(theta, m),
                      _c_m_closed(theta, m))


def cm_table(theta, l, eps=1e-4):
    """Vector [c₀ … c_{l+1}]ᵀ using the safe formula."""
    return ca.vcat([c_m_safe(theta, m, eps) for m in range(l + 2)])


# -----------------------------------------------------------------------------
# 2.  factory for  P(Ω, A)  with truncation index nil_deg   (SX symbols only)
# -----------------------------------------------------------------------------
def P(omega, A, B, nil_deg):
    """
    Parameters
    ----------
    B : nil-potent square matrix
    nil_deg: nilpotency degree of B
    Returns
    -------
    ca.Function   P(theta, Omega, A, B) -> n×n SX matrix
    """


    # ---------- symbolic inputs ----------
    Omega = ca.skew(omega)
    theta = ca.norm_2(omega)
    n = B.shape[0]

    # ---------- scalar coefficient tables ----------
    alpha = [1.0 / math.factorial(m + 1) for m in range(nil_deg)]   # 1/(m+1)!
    cm = cm_table(theta, nil_deg)                                  # [c₀ … c_{l+1}]

    # ---------- shared‑power loop ----------
    S0 = ca.SX.zeros(n, n)
    S1 = ca.SX.zeros(n, n)
    S2 = ca.SX.zeros(n, n)
    T  = ca.SX.eye(n) # B⁰

    for m in range(nil_deg):
        S0 += alpha[m] * T
        S1 += cm[m] * T
        S2 += cm[m+1] * T
        T = T @ B

    OmegaA = Omega@A

    # ---------- assemble P ----------
    return A @ S0 + OmegaA @ S1 + Omega @ OmegaA @ S2

# -----------------------------------------------------------------------------
# 3.  quick demo
# -----------------------------------------------------------------------------
if __name__ == '__main__':
    omega = np.array([0.1, 0.2, 0.3], float)
    B     = np.array([[0,1,0],[0,0,0],[0,0,0]], float)
    A     = np.random.rand(3, B.shape[0])
    P0 = P(omega, A, B, 1)
    print("P =", P0)

P = 
[[0.0768327, 0.682552, 0.625768], 
 [0.299501, 0.587554, 0.480925], 
 [0.0887573, 0.910165, 0.960526]]


In [2]:
from cyecca import lie

def exp_mixed_old(
    self,
    X0: lie.group_se23.SE23LieGroupElement,
    l: lie.group_se23.SE23LieAlgebraElement,
    r: lie.group_se23.SE23LieAlgebraElement,
    B: ca.SX,
) -> lie.group_se23.SE23LieGroupElement:
    n = B.shape[0]
    P0 = ca.horzcat(X0.v.param, X0.p.param)
    Pl = self.calculate_N(l, B)
    Pr = self.calculate_N(r, -B)
    R0 = X0.R
    Rl = (l).Omega.exp(self.SO3)
    Rr = (r).Omega.exp(self.SO3)
    Rr0 = Rr * R0
    R1 = Rr0 * Rl

    I = ca.SX.eye(n)
    P1 = Rr0.to_Matrix() @ Pl + (Rr.to_Matrix() @ P0 + Pr) @ (I + B)
    return self.elem(ca.vertcat(P1[:, 1], P1[:, 0], R1.param))

def exp_mixed_new(
    self,
    X0: lie.group_se23.SE23LieGroupElement,
    l: lie.group_se23.SE23LieAlgebraElement,
    r: lie.group_se23.SE23LieAlgebraElement,
    B: ca.SX,
    nil_deg: int,
) -> lie.group_se23.SE23LieGroupElement:
    n = B.shape[0]
    P0 = ca.horzcat(X0.v.param, X0.p.param)
    
    Al = ca.horzcat(l.param[3:6], l.param[6:9])
    Ar = ca.horzcat(r.param[3:6], r.param[6:9])

    Pl = P(l.param[:3], Al, B, nil_deg)
    Pr = P(r.param[:3], Ar, -B, nil_deg)

    R0 = X0.R
    Rl = (l).Omega.exp(self.SO3)
    Rr = (r).Omega.exp(self.SO3)
    Rr0 = Rr * R0
    R1 = Rr0 * Rl

    I = ca.SX.eye(n)
    P1 = Rr0.to_Matrix() @ Pl + (Rr.to_Matrix() @ P0 + Pr) @ (I + B)

    return self.elem(ca.vertcat(P1[:, 1], P1[:, 0], R1.param))

def find_flops(op_dict):
    flops = 0
    for k, v in op_dict.items():
        if k not in ["OP_PARAMETER", "OP_CONST"]:
            flops += v
    return flops

In [3]:
dt = ca.SX.sym("dt")
X0 = lie.SE23Mrp.elem(ca.SX.sym("X0", 9))
a_b = ca.SX.sym("a_b", 3)
g = ca.SX.sym("g")
omega_b = ca.SX.sym("omega_b", 3)
l = lie.se23.elem(ca.vertcat(0, 0, 0, a_b, omega_b))
r = lie.se23.elem(ca.vertcat(0, 0, 0, 0, 0, g, 0, 0, 0))
B = ca.sparsify(ca.SX([[0, 1], [0, 0]]))

#X1 = exp_mixed_old(lie.SE23Mrp, X0, l * dt, r * dt, B * dt)
#X1

X1 = exp_mixed_new(lie.SE23Mrp, X0, l * dt, r * dt, B * dt, 2)
X1


SE23LieGroup: SX(@1=1, @2=8, @3=((sq(X0_6)+sq(X0_7))+sq(X0_8)), @4=sq((@1+@3)), @5=(@1-((@2*(sq(X0_8)+sq(X0_7)))/@4)), @6=(dt*a_b_0), @7=0.5, @8=(@7*dt), @9=(dt*omega_b_0), @10=((@6*@8)+@9), @11=(4*(@1-@3)), @12=(((@2*(X0_6*X0_7))-(@11*X0_8))/@4), @13=(dt*a_b_1), @14=(dt*omega_b_1), @15=((@13*@8)+@14), @16=(((@2*(X0_6*X0_8))+(@11*X0_7))/@4), @17=(dt*a_b_2), @18=(dt*omega_b_2), @19=((@17*@8)+@18), @20=(((@2*(X0_7*X0_6))+(@11*X0_8))/@4), @21=(@1-((@2*(sq(X0_8)+sq(X0_6)))/@4)), @22=(((@2*(X0_7*X0_8))-(@11*X0_6))/@4), @23=(((@2*(X0_8*X0_6))-(@11*X0_7))/@4), @24=(((@2*(X0_8*X0_7))+(@11*X0_6))/@4), @25=(@1-((@2*(sq(X0_7)+sq(X0_6)))/@4)), @26=(dt*g), @27=(X0_5+@26), @28=((sq(X0_6)+sq(X0_7))+sq(X0_8)), @29=(@1-@28), @30=((sq(@9)+sq(@14))+sq(@18)), @31=(fabs(@30)<0.001), @32=0.25, @33=((@31?(((((@32+(0.00520833*@30))+(0.000130208*sq(@30)))+(3.29396e-06*(@30*sq(@30))))+(8.34255e-08*sq(sq(@30))))+(2.11316e-09*(@30*sq(sq(@30))))):0)+((!@31)?(pow(@30,-0.5)*tan((@32*sqrt(@30)))):0)), @34=(@33*@9), @

In [4]:
from cyecca import lie

def derive_strapdown_ins_propagation_old():
    dt = ca.SX.sym("dt")
    X0 = lie.SE23Mrp.elem(ca.SX.sym("X0", 9))
    a_b = ca.SX.sym("a_b", 3)
    g = ca.SX.sym("g")
    omega_b = ca.SX.sym("omega_b", 3)
    l = lie.se23.elem(ca.vertcat(0, 0, 0, a_b, omega_b))
    r = lie.se23.elem(ca.vertcat(0, 0, 0, 0, 0, g, 0, 0, 0))
    B = ca.sparsify(ca.SX([[0, 1], [0, 0]]))
    X1 = exp_mixed_old(lie.SE23Mrp, X0, l * dt, r * dt, B * dt)
    r1 = lie.SO3Mrp.elem(X1.param[6:9])
    lie.SO3Mrp.shadow_if_necessary(r1)
    X1.param[6:9] = r1.param
    return ca.Function(
        "strapdown_ins_propagate",
        [X0.param, a_b, omega_b, g, dt],
        [X1.param],
        ["x0", "a_b", "omega_b", "g", "dt"],
        ["x1"],
    )

def derive_strapdown_ins_propagation_new():
    dt = ca.SX.sym("dt")
    X0 = lie.SE23Mrp.elem(ca.SX.sym("X0", 9))
    a_b = ca.SX.sym("a_b", 3)
    g = ca.SX.sym("g")
    omega_b = ca.SX.sym("omega_b", 3)
    l = lie.se23.elem(ca.vertcat(0, 0, 0, a_b, omega_b))
    r = lie.se23.elem(ca.vertcat(0, 0, 0, 0, 0, g, 0, 0, 0))
    B = ca.sparsify(ca.SX([[0, 1], [0, 0]]))
    X1 = exp_mixed_new(lie.SE23Mrp, X0, l * dt, r * dt, B * dt, 2)
    r1 = lie.SO3Mrp.elem(X1.param[6:9])
    lie.SO3Mrp.shadow_if_necessary(r1)
    X1.param[6:9] = r1.param
    return ca.Function(
        "strapdown_ins_propagate",
        [X0.param, a_b, omega_b, g, dt],
        [X1.param],
        ["x0", "a_b", "omega_b", "g", "dt"],
        ["x1"],
    )

sins_old = derive_strapdown_ins_propagation_old()
sins_new = derive_strapdown_ins_propagation_new()

In [5]:
from cyecca.util import count_ops
omega = ca.SX.sym('omega', 3)
A = ca.SX.sym('A', 3, B.shape[0])

sins_sym_old = sins_old(
    ca.SX.sym("x0", 9),
    ca.SX.sym("a_b", 3),
    ca.SX.sym("omega_b", 3),
    ca.SX.sym("g", 1),
    ca.SX.sym("dt", 1),
)

sins_sym_new = sins_new(
    ca.SX.sym("x0", 9),
    ca.SX.sym("a_b", 3),
    ca.SX.sym("omega_b", 3),
    ca.SX.sym("g", 1),
    ca.SX.sym("dt", 1),
)


In [6]:
op_dict_old = count_ops(sins_sym_old)
flops_old = find_flops(op_dict_old)
flops_old

building dependency graph...done


414

In [7]:
op_dict_new = count_ops(sins_sym_new)
flops_new = find_flops(op_dict_new)
flops_new

building dependency graph...done


256

In [60]:
x0 = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])
a_b = np.array([0, 0, 0])
omega_b = np.array([0, 0, 0])
g = 9.8
dt = 0.01
sins_old(x0, a_b, omega_b, g, dt)

DM([1.04, 2.05, 3.06049, 4, 5, 6.098, -0.0360825, -0.0412371, -0.0463918])

In [61]:
sins_new(x0, a_b, omega_b, g, dt)

DM([1.04, 2.05, 3.06049, 4, 5, 6.098, -0.0360825, -0.0412371, -0.0463918])